In [12]:
from smolagents import LiteLLMModel, ToolCallingAgent
import os

model = LiteLLMModel(model_id="gemini/gemini-2.0-flash-lite", api_key=os.getenv('GEMINI_API_KEY'))

In [6]:
import re
import requests
from markdownify import markdownify as md
from requests.exceptions import RequestException
from smolagents import tool

@tool
def visit_webpage(url: str) -> str:
    """
    Visits a webpage at the given URL and returns its content as a markdown string.

    Args:
        url (str): The URL of the webpage to visit.

    Returns:
        str: The content of the webpage converted to markdown, or an error message if the request fails.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()

        markdown_content = md(response.text).strip()
        markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)

        return markdown_content

    except RequestException as e:
        return f"Error fetching the webpage: {str(e)}"
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"


In [8]:
print(visit_webpage("https://en.wikipedia.org/wiki/Football_Manager")[:5000])

Football Manager - Wikipedia

[Jump to content](#bodyContent)

Main menu

Main menu

move to sidebar
hide

Navigation

* [Main page](/wiki/Main_Page "Visit the main page [z]")
* [Contents](/wiki/Wikipedia:Contents "Guides to browsing Wikipedia")
* [Current events](/wiki/Portal:Current_events "Articles related to current events")
* [Random article](/wiki/Special:Random "Visit a randomly selected article [x]")
* [About Wikipedia](/wiki/Wikipedia:About "Learn about Wikipedia and how it works")
* [Contact us](//en.wikipedia.org/wiki/Wikipedia:Contact_us "How to contact Wikipedia")

Contribute

* [Help](/wiki/Help:Contents "Guidance on how to use and edit Wikipedia")
* [Learn to edit](/wiki/Help:Introduction "Learn how to edit Wikipedia")
* [Community portal](/wiki/Wikipedia:Community_portal "The hub for editors")
* [Recent changes](/wiki/Special:RecentChanges "A list of recent changes to Wikipedia [r]")
* [Upload file](/wiki/Wikipedia:File_upload_wizard "Add images or other media for use o

In [14]:
from smolagents import ToolCallingAgent, DuckDuckGoSearchTool

web_agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool(), visit_webpage],
    model=model
)

/Users/ahaanshah/Desktop/Duke/ECs/summer-25/agentic-rag/.venv/lib/python3.13/site-packages/smolagents/default_tools.py:135: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  self.ddgs = DDGS(**kwargs)


In [15]:
managed_web_agent = ManagedAgent(
    agent=web_agent,
    name="search_agent",
    description="Runs web searches for you. Give it your query as an argument.",
)

NameError: name 'ManagedAgent' is not defined